In [252]:
import SimFunctions
import SimRNG 
import SimClasses
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import scipy.stats as stats

In [253]:
import xlrd
# Read the data file we got through data cleaning. 
wb = xlrd.open_workbook('BikeRentCount.xlsx')
# The file consists of five sheets and each sheet contains data for one station.
sheets = wb.sheet_names()
MaxRentingRate=[]
RentingRate=[]
for i in range(len(sheets)):
    data = pd.read_excel('BikeRentCount.xlsx', sheet_name=sheets[i])
    #Calculate the average number of rented bikes for each twenty minutes of each station.
    rentingRates = data.mean()
    RentingRate.append(rentingRates)
    # Find the maximum of these rates that would be used for function NSPP
    maxRentingRate = max(rentingRates)
    MaxRentingRate.append(maxRentingRate)
    
# Example: the number of rented bikes at Station 2 for every 20 minutes (indexing from 0) 
RentingRate[1]

7:00-7:20     1.869565
7:20-7:40     4.967391
7:40-8:00     7.923913
8:00-8:20     4.619565
8:20-8:40     6.086957
8:40-9:00     4.717391
9:00-9:20     3.478261
9:20-9:40     3.217391
9:40-10:00    1.706522
dtype: float64

In [254]:
# Example: the number of rented bikes at Station 2(indexing from 0) during time 8:40-9:00
RentingRate[1][5]

4.717391304347826

In [255]:
# Example: the maximum number of rented bikes at station 2(indexing from 0)
MaxRentingRate[1]

7.923913043478261

In [256]:
# Similar data processing for 'BikeReturnCount.xlsx'
wb = xlrd.open_workbook('BikeReturnCount.xlsx')
sheets = wb.sheet_names()
MaxReturnRate=[]
ReturnRate=[]
for i in range(len(sheets)):
    data = pd.read_excel('BikeReturnCount.xlsx', sheet_name=sheets[i])
    returnRates = data.mean()
    ReturnRate.append(returnRates)
    maxReturnRate = max(returnRates)
    MaxReturnRate.append(maxReturnRate)

# Simulation constants

In [318]:
order_threshold = 5.0 #schedule the "rebalancing" once the number of bikes is less than the threshold
order_up_to = 10.0 # the number of bikes in the station would reach 5 after rebalancing
delivery_delay = 20 # it takes 20 minutes for bikes to be transported to the station for bike rebalancing
SIM_RUN = 10 #number of simulation runs
operation_cost = 2 # USD per bike for operation
oil_gas = 3 # USD per 1 refillment
service_fee = 3.75 # USD per bike per ride
PENALTY = 3 # CAD for cost of loss of business oportunity
loss_profit = 0.01 # CAD per bike per minute for loss of business opportunity
RunLength = 180 # 3 hours

In [350]:
def Rental(**kwargs):
    global revenue, penalty, Loss_profit
    #Check which station the rental event occurs in
    station_id = kwargs['stationID']
    Pickup_queue = kwargs["pickup_queue"]
    Dropoff_queue = kwargs["dropoff_queue"]
    #Use NSPP to schedule the next bike rental event for current station
    interarrival = NSPP(station_id,MaxRentingRate,RentingRate)
    SimFunctions.Schedule(Calendar,"Rent a bike",interarrival,stationID = station_id,pickup_queue = TheQueues[station_id],dropoff_queue = TheQueues[station_id+5])

    # Checks if there are people waiting to put the bikes back.
    if Dropoff_queue.NumQueue() > 0:
        DepartingCustomer = Dropoff_queue.Remove()
        Waiting_time = SimClasses.Clock - DepartingCustomer.CreateTime
        Wait.Record(SimClasses.Clock - DepartingCustomer.CreateTime)
        # If customer has waited too long to return the bike, refund is given.
        # The customer gets the bike from the people waiting to put the bikes back.
        if Waiting_time >5:
            penalty += service_fee
            
    #If no one is waiting to put the bikes back,
    #check if there are bikes available and update number of bikes for the station.
    else:
        if Num_bikes[station_id]>0:
            Num_bikes[station_id]-=1
            # Customer pays to rent bike.
            revenue += service_fee
        #No bikes available so the customer begins waiting for bikes to become available.
        else:
            Customer = SimClasses.Entity()
            Pickup_queue.Add(Customer)
            
    #Schedule the bike rebalancing event once the number of bikes is less than the threshold 
    """
    There are two conditions to be met for the rebalancing operation. Schedule the rebalancing operation 
    once the number of bikes is less than the threshold and the previous rebalance operation has ended. 
    """
    if Num_bikes[station_id] <= order_threshold and quantity[station_id] == 0:
        quantity[station_id] = order_up_to - Num_bikes[station_id]
        SimFunctions.Schedule(Calendar,"rebalancing",delivery_delay,stationID=station_id,pickup_queue = TheQueues[station_id],dropoff_queue = TheQueues[station_id+5],num_ordered=quantity[station_id])

In [331]:
def RideEnd(**kwargs):
    global  revenue, penalty, Loss_profit
    station_id = kwargs['stationID']
    Dropoff_queue = kwargs['dropoff_queue']
    Pickup_queue = kwargs['pickup_queue']
     ##Use NSPP to schedule the next end of ride for current station
    interarrival = NSPP(station_id,MaxReturnRate,ReturnRate)
    SimFunctions.Schedule(Calendar,"Return a bike",interarrival,stationID = station_id,pickup_queue = TheQueues[station_id],dropoff_queue = TheQueues[station_id+5])
    
    # We assume not every customer will wait for a bike and eventually take a bike.Customers leave after 5 mins.
    while(True):
        # Check if customers are waiting.
        if Pickup_queue.NumQueue() == 0:
            #If no one is waiting to pick up the bike,check if there are empty racks to keep the bike.   
            if Num_bikes[station_id] < Num_docks[station_id]:
                # Customer returns the bike to the rack.
                Num_bikes[station_id] += 1
            # No empty racks. The customer begins waiting in queue.
            else:
                Customer = SimClasses.Entity()
                Dropoff_queue.Add(Customer)
            break
        #If there are customers waiting to pick up the bike,
        #check if the customer has left due to waiting too long to pick up the bike.
        DepartingCustomer = Pickup_queue.Remove()
        Waiting_time = SimClasses.Clock - DepartingCustomer.CreateTime
        Wait.Record(SimClasses.Clock - DepartingCustomer.CreateTime)
        if Waiting_time < 5:
            # Next waiting customer gets a bike from the customer who has finished the ride.
            # Break the while loop.
            break
        else:
            # We lose a customer because the customer has waited too long
            penalty += PENALTY

In [332]:
def rebalancing(**kwargs):
    global revenue , cost, Loss_profit,penalty
    station_id = kwargs['stationID']
    Dropoff_queue = kwargs['dropoff_queue']
    Pickup_queue = kwargs['pickup_queue']
    #The number of bikes needed for bike rebalancing
    num_ordered = kwargs['num_ordered']
    #Cost due to bike rebalanccing
    cost += (num_ordered * operation_cost) + oil_gas 
    #The number of bikes changes after the rebalancing operation
    Num_bikes[station_id] += num_ordered
    # We assume not every customer will wait for a bike and eventually take a bike..Customers leave after 5 mins.
    while(True):
        # Check if customers are waiting in the pick_up queue and if the ordered number of bikes is in short supply.
        if (Pickup_queue.NumQueue() == 0.0) | (Num_bikes[station_id] == 0.0):
            break

        DepartingCustomer = Pickup_queue.Remove()
        Waiting_time = SimClasses.Clock - DepartingCustomer.CreateTime
        Wait.Record(SimClasses.Clock - DepartingCustomer.CreateTime)
        if Waiting_time < 5:
            # Next waiting customer gets a bike
             Num_bikes[station_id] -= 1
        else:
            # We lose a customer
            penalty += PENALTY

    #Set the num_oredered to zero so the current rebalancing operation is over.
    quantity[station_id] = 0

In [333]:
# Prevent index out of range

def PW_ArrRate(p,t,Rates):
    hour = int(t/20)
    if hour <= 8:
        return Rates[p][hour]
    else:
        return Rates[p][-1]


In [334]:
def NSPP(q,MaxRate,Rate):
    PossibleArrival = SimClasses.Clock + SimRNG.Expon(1/(MaxRate[q]/20), 1)
    while SimRNG.Uniform(0, 1, 1) >= PW_ArrRate(q,PossibleArrival,Rate)/(MaxRate[q]):
        PossibleArrival = PossibleArrival + SimRNG.Expon(1/(MaxRate[q]/20), 1)
    nspp = PossibleArrival - SimClasses.Clock
    return nspp

In [335]:
# Get all trial solution 
trial_solution = []
for a in range(5,15):
    for b in range(5,27):
        for c in range(5,11):
            for d in range(5,15):
                for e in range(5,15):
                    # Initial number of bikes available in the system.
                    if (a+b+c+d+e)==70:
                        trial_solution.append([a,b,c,d,e])
len(trial_solution)

480

In [351]:
Calendar = SimClasses.EventCalendar()
Wait = SimClasses.DTStat()

Pickup_Queue1 = SimClasses.FIFOQueue()
Pickup_Queue2 = SimClasses.FIFOQueue()
Pickup_Queue3 = SimClasses.FIFOQueue()
Pickup_Queue4 = SimClasses.FIFOQueue()
Pickup_Queue5 = SimClasses.FIFOQueue()

Dropoff_Queue1 = SimClasses.FIFOQueue()
Dropoff_Queue2 = SimClasses.FIFOQueue()
Dropoff_Queue3 = SimClasses.FIFOQueue()
Dropoff_Queue4 = SimClasses.FIFOQueue()
Dropoff_Queue5 = SimClasses.FIFOQueue()

TheCTStats = []
TheDTStats = []
TheQueues = []
TheResources = []

TheQueues.append(Pickup_Queue1)
TheQueues.append(Pickup_Queue2)
TheQueues.append(Pickup_Queue3)
TheQueues.append(Pickup_Queue4)
TheQueues.append(Pickup_Queue5)
TheQueues.append(Dropoff_Queue1)
TheQueues.append(Dropoff_Queue2)
TheQueues.append(Dropoff_Queue3)
TheQueues.append(Dropoff_Queue4)
TheQueues.append(Dropoff_Queue5)

TheDTStats.append(Wait)

AllWaitMean = np.zeros((SIM_RUN,len(trial_solution)))

ZSimRNG = SimRNG.InitializeRNSeed()

#Create an 2d array to record the balance of each trial solution in each simulation run
output = np.zeros((SIM_RUN,len(trial_solution)))
for k in range(SIM_RUN):
    for j in range(len(trial_solution)):
        Num_bikes=[]
        initial_Num_bikes=[]
        #Apply the our model to each trial solution
        for i in range(0,5,1):
            Num_bikes.append(trial_solution[j][i])
            initial_Num_bikes.append(trial_solution[j][i])
        # Number of bike racks (total) at each station. This is the maximum parking
        # capacity for every station.
        Num_docks=[15,27,11,15,15]
        #initial number of ordered bikes for rebalancing
        quantity=[0,0,0,0,0]
        revenue = 0
        cost = 0
        penalty = 0
        Loss_profit = 0

        SimClasses.Clock = 0.0
        SimFunctions.SimFunctionsInit(Calendar,TheQueues,TheCTStats,TheDTStats,TheResources)
        for i in range(0,5,1):
            #Use NSPP to schedule the first arrival for each station.
            #Initialize queues at each station.
            SimFunctions.Schedule(Calendar,"Rent a bike",NSPP(i,MaxRentingRate,RentingRate),stationID = i,pickup_queue = TheQueues[i],dropoff_queue = TheQueues[i+5])
            SimFunctions.Schedule(Calendar,"Return a bike",NSPP(i,MaxReturnRate,ReturnRate),stationID = i,pickup_queue = TheQueues[i],dropoff_queue = TheQueues[i+5]) 
        SimFunctions.Schedule(Calendar,"EndSimulation",RunLength)
        NextEvent = Calendar.Remove()
        SimClasses.Clock = NextEvent.EventTime
        if NextEvent.EventType == "Rent a bike":
            Rental(**NextEvent.kwargs)
        elif NextEvent.EventType == "Return a bike":
            RideEnd(**NextEvent.kwargs)
        elif NextEvent.EventType == "rebalancing":
            rebalancing(**NextEvent.kwargs)

        while NextEvent.EventType != "EndSimulation":
            NextEvent = Calendar.Remove()
            SimClasses.Clock = NextEvent.EventTime
            if NextEvent.EventType == "Rent a bike":
                Rental(**NextEvent.kwargs)
            elif NextEvent.EventType == "Return a bike":
                RideEnd(**NextEvent.kwargs)
            elif NextEvent.EventType == "rebalancing":
                rebalancing(**NextEvent.kwargs)
        repositioning_bikes= (abs(np.array(Num_bikes)-np.array(initial_Num_bikes)).sum())
        cost += (repositioning_bikes * operation_cost) + oil_gas 
        balance = revenue - cost - penalty - Loss_profit
        output[k][j]= balance
        AllWaitMean[k][j] = Wait.Mean()




In [352]:
output

array([[187.25, 151.  , 170.5 , ..., 200.5 , 158.  , 195.25],
       [155.75, 120.5 , 177.  , ..., 161.75, 146.75, 177.25],
       [143.  , 158.5 , 173.  , ...,  99.  , 159.25, 158.  ],
       ...,
       [180.75, 120.25, 162.  , ..., 193.75, 145.5 , 146.25],
       [122.75, 165.  , 134.5 , ..., 209.5 , 157.  , 187.5 ],
       [154.25, 136.75, 148.5 , ..., 135.  , 141.25, 143.25]])

In [356]:
AllWaitMean

array([[ 0.        , 18.7115286 , 15.37531993, ...,  0.        ,
        21.96521418,  4.61418236],
       [ 0.        , 21.23296163, 12.40094521, ..., 34.02634907,
        25.04008151,  9.26252572],
       [13.80984385, 11.88478011,  1.79072886, ..., 18.92836138,
        14.58184105,  9.34044501],
       ...,
       [37.17612842, 35.66504066, 16.43514841, ...,  2.11240261,
        32.34306876,  0.        ],
       [50.46385118, 29.49521446, 21.16419971, ...,  0.        ,
         2.03539243,  2.85539226],
       [14.48166639, 18.8992971 , 69.33925762, ..., 11.8653238 ,
         9.02600328, 15.8995623 ]])

In [353]:
#Get the the expectation by calculating the mean over all simulation runs for each trial solution
Estimated_Expected_waittime=np.mean(AllWaitMean,axis=0)
#Get the best solution to minimize customers waiting time. 
trial_solution[np.argmin(Estimated_Expected_waittime)]

[14, 25, 9, 12, 10]

In [354]:
#Get the the expectation by calculating the mean over all simulation runs for each trial solution
Estimated_Expected_balance=np.mean(output,axis=0)
#Get the best solution to maximize the balance.
trial_solution[np.argmax(Estimated_Expected_balance)]

[13, 21, 9, 14, 13]

In [322]:
#the estimated expected balance of all trial solutions
Estimated_Expected_balance

array([181.575, 187.525, 190.425, 199.45 , 186.225, 187.925, 200.95 ,
       225.95 , 184.025, 197.825, 209.85 , 188.1  , 198.025, 199.2  ,
       194.25 , 209.275, 201.275, 209.25 , 198.55 , 204.6  , 201.75 ,
       211.425, 215.85 , 197.325, 205.1  , 210.675, 197.95 , 196.325,
       181.525, 177.9  , 212.525, 218.2  , 202.175, 222.9  , 182.375,
       203.6  , 196.15 , 206.775, 194.025, 186.25 , 206.925, 209.45 ,
       200.05 , 219.175, 206.25 , 216.075, 209.525, 220.325, 222.825,
       223.25 , 214.7  , 230.475, 199.525, 210.925, 204.775, 181.15 ,
       180.35 , 192.425, 191.1  , 204.9  , 204.45 , 223.1  , 210.625,
       216.075, 196.475, 200.975, 211.875, 223.475, 228.75 , 199.825,
       201.775, 198.75 , 216.375, 203.15 , 203.275, 211.675, 208.85 ,
       218.55 , 221.125, 199.075, 200.725, 227.05 , 186.55 , 185.575,
       217.95 , 190.925, 230.975, 227.65 , 219.8  , 220.7  , 208.575,
       207.25 , 192.425, 220.5  , 194.5  , 199.925, 222.15 , 215.75 ,
       221.1  , 194.